In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
import pyautogui as py


def data_collection(xpath_route):
    route = driver.find_element(By.XPATH, xpath_route)
    driver.execute_script("arguments[0].scrollIntoView(true);", route)
    time.sleep(2)
    path = route.text
    print(f"started to fetch the bus data for {path}")

    route.click()

    try:
        driver.execute_script("window.scrollTo(0, 0);")
        time.sleep(1)
        route = driver.find_element(By.XPATH, '//*[@id="result-section"]/div[3]/div/div[2]/div/div[4]/div[2]')
        driver.execute_script("arguments[0].scrollIntoView(true);", route)

        # Getting the window height 
        window_height = driver.execute_script("return window.innerHeight")

        # Scroll by 20% of the window height 
        driver.execute_script(f"window.scrollBy(0, {-window_height * 0.3});")

        time.sleep(1)
        route.click()

    except:
        pass


    try:
        driver.execute_script("window.scrollTo(0, 0);")
        time.sleep(1)
        route = driver.find_element(By.XPATH, '//*[@id="result-section"]/div[2]/div/div[2]/div/div[4]/div[2]')
        driver.execute_script("arguments[0].scrollIntoView();", route)

        # Getting the window height 
        window_height = driver.execute_script("return window.innerHeight")

        # Scroll by 20% of the window height using JavaScript
        driver.execute_script(f"window.scrollBy(0, {-window_height * 0.3});")

        time.sleep(1)
        route.click()

    except:
        pass

    try:
        driver.execute_script("window.scrollTo(0, 0);")
        time.sleep(1)
        route = driver.find_element(By.XPATH, '//*[@id="result-section"]/div[1]/div/div[2]/div/div[4]/div[2]')
        driver.execute_script("arguments[0].scrollIntoView(true);", route)
        # Getting the window height 
        window_height = driver.execute_script("return window.innerHeight")

        # Scroll by 20% of the window height 
        driver.execute_script(f"window.scrollBy(0, {-window_height * 0.3});")

        time.sleep(1)
        route.click()


    except:
        pass


    bus_details = []
#     bus_details = {"name":[], "type":[], "departure_time":[], "arrival_time":[], "duration":[], "price":[], "seats_available":[], "rating":[]}

    element = WebDriverWait(driver, 30).until(EC.presence_of_all_elements_located((By.XPATH,'//div[@class="clearfix bus-item"]')))
    body = driver.find_element(By.TAG_NAME, 'body')
            
    # Getting the initial scroll height
    last_height = driver.execute_script("return document.body.scrollHeight")

    while True:
        # Scrolling down to the bottom of the page
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # Wait for 0.5 seconds
        time.sleep(1.5)

        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height


    buses = driver.find_elements(By.XPATH,'//div[@class="clearfix bus-item-details"]')

    for bus in buses:
        obj = bus.find_element(By.CSS_SELECTOR, ".seat-left")
        driver.execute_script("arguments[0].scrollIntoView(true);", obj)

        # Getting the window height 
        window_height = driver.execute_script("return window.innerHeight")

        # Scroll by 20% of the window height 
        driver.execute_script(f"window.scrollBy(0, -{window_height * 0.3});")

        #Extract various attributes of the bus using CSS selectors

        name = bus.find_element(By.CSS_SELECTOR, ".travels").text
        type = bus.find_element(By.CSS_SELECTOR,".bus-type").text
        departure_time = bus.find_element(By.CSS_SELECTOR, ".dp-time").text
        arrival_time = bus.find_element(By.CSS_SELECTOR, ".bp-time").text
        duration = bus.find_element(By.CSS_SELECTOR, ".dur").text
        price = bus.find_element(By.CSS_SELECTOR, ".fare").text
        seats_available=bus.find_element(By.CSS_SELECTOR,".seat-left").text

        #Handle the possibility that the rating might not be available
        try:
            rating = bus.find_element(By.CSS_SELECTOR, ".rating-sec .rating").text
        except:
            rating = "No rating"

        #Append the extracted details to the bus_details lisT    
        bus_details.append({
            "name": name,
            "type":type,
            "departure_time": departure_time,
            "arrival_time": arrival_time,
            "duration": duration,
            "price": price,
            "seats_available":seats_available,
            "rating": rating
        })


    bus_data = {"route" : [], "name" : [], "type" : [], "departure_time" : [], "arrival_time" : [], "duration" : [], "price" : [], "seats_available" : [], "rating" : []}
#Populate the bus_data dictionary with details from bus_details

    for bus in bus_details:
        bus_data["name"].append(bus["name"])
        bus_data["type"].append(bus["type"])
        bus_data["departure_time"].append(bus["departure_time"])
        bus_data["arrival_time"].append(bus["arrival_time"])
        bus_data["duration"].append(bus["duration"])
        bus_data["price"].append(bus["price"])
        bus_data["seats_available"].append(bus["seats_available"])
        bus_data["rating"].append(bus["rating"])
        bus_data["route"].append(path)

    bus_df = pd.DataFrame(bus_data)

    driver.quit()
    

    print(f"successfully fetched {(bus_df).shape[0]} no of buses data for route {path}")
    print("\n")
    
    return bus_df

In [1]:
#Extracting datas using url

state_path = 'https://www.redbus.in/online-booking/hrtc'
state = "Himachal_Pradesh"
bus_list = []

#XPath to locate the route section on the page
driver = webdriver.Chrome()
driver.get(state_path)
xpath_route = f'//*[@id="root"]/div/div[4]/div[12]'
route = driver.find_element(By.XPATH, xpath_route)
driver.execute_script("arguments[0].scrollIntoView(true);", route)
pages = (route.text.split('\n'))
page_numbers = []

#Iterate through each page number
for i in pages:
    page_numbers.append(int(i))
    
for page in page_numbers:

    if page == 1:
        for i in range(2,15):
            try:
                data = ""
                driver = webdriver.Chrome()
                driver.get(state_path)
                xpath_route = f'//*[@id="root"]/div/div[4]/div[{i}]/div[1]/a'
                route = driver.find_element(By.XPATH, xpath_route)
                driver.execute_script("arguments[0].scrollIntoView(true);", route)
                data = data_collection(xpath_route)
                bus_list.append(data)

            except:
                continue

    else:
        for i in range(2,15):
            try:
                data = ""
                driver = webdriver.Chrome()
                driver.get(state_path)

                xpath_page = f'//*[@id="root"]/div/div[4]/div[12]/div[{page}]'
                route = driver.find_element(By.XPATH, xpath_page)
                driver.execute_script("arguments[0].scrollIntoView(true);", route)
                time.sleep(1)
                route.click()

                xpath_route = f'//*[@id="root"]/div/div[4]/div[{i}]/div[1]/a'

                data = data_collection(xpath_route)
                bus_list.append(data)
            except:
                continue
            
driver.quit()

NameError: name 'webdriver' is not defined

In [13]:
bus_list

[          route                                               name  \
 0   Pune to Goa  Kadamba Transport Corporation Limited (KTCL) -...   
 1   Pune to Goa                                      Atmaram Gobus   
 2   Pune to Goa                                       Zingbus Plus   
 3   Pune to Goa                                      Atmaram Gobus   
 4   Pune to Goa                                 Ashray Vip Yellows   
 5   Pune to Goa                                     Ashray Travels   
 6   Pune to Goa                                       Zingbus Plus   
 7   Pune to Goa                                       Zingbus Plus   
 8   Pune to Goa                                     Ashray Travels   
 9   Pune to Goa                                      Atmaram Gobus   
 10  Pune to Goa                                Laxmi-Princess Nova   
 11  Pune to Goa                                    Ashray Amrutyog   
 12  Pune to Goa                              Prasanna - Purple Bus   
 13  P

In [14]:
#Concatenate all dataframes in the 'bus_list' into a single dataframe using state variables
import pandas as pd
try:
    df = pd.concat(bus_list)
    df.to_csv(f"{state}_buses.csv", index = False)
except:
    bus_data = {"route" : [], "name" : [], "type" : [], "departure_time" : [], "arrival_time" : [], "duration" : [], "price" : [], "seats_available" : [], "rating" : []}
    df = pd.DataFrame(bus_data)
    df.to_csv(f"{state}_buses.csv", index = False)